# Assignment 7: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A7-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [1]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

'0.13.3'

In [2]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")
print(users["label"].value_counts())
orders.shape

False    628
True     139
Name: label, dtype: int64


(5997, 4)

## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [3]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    entities = {"users":(users,"user_id"),
               "orders":(orders,"order_id","order_time"),
               "order_products":(order_products,"order_product_id","order_time")}
    relationships = [("users","user_id","orders","user_id"),("orders","order_id","order_products","order_id")]
    ent_set = ft.EntitySet("retail",entities,relationships)
    
    return ent_set

### 1.2 Deep Feature Synthesis

In [4]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity="users")

In [5]:
# output what features you generate
feature_matrix.columns
feature_matrix = pd.get_dummies(feature_matrix)
feature_matrix = feature_matrix.fillna(0)
feature_matrix

,Unnamed: 0,label,SUM(orders.Unnamed: 0),STD(orders.Unnamed: 0),MAX(orders.Unnamed: 0),SKEW(orders.Unnamed: 0),MIN(orders.Unnamed: 0),MEAN(orders.Unnamed: 0),COUNT(orders),SUM(order_products.reordered),...,MODE(orders.MODE(order_products.product_name))_Organic Reduced Fat Omega-3 Milk,MODE(orders.MODE(order_products.product_name))_Organic Tortilla Chips,MODE(orders.MODE(order_products.product_name))_Original Beef Jerky,MODE(orders.MODE(order_products.product_name))_Red Plastic Cups,MODE(orders.MODE(order_products.product_name))_Roasted Pine Nut Hummus,MODE(orders.MODE(order_products.product_name))_Roasted Seaweed Snacks,MODE(orders.MODE(order_products.product_name))_Smartwater,"MODE(orders.MODE(order_products.product_name))_Sparkling Water, Bottles",MODE(orders.MODE(order_products.product_name))_Spring Water,MODE(orders.MODE(order_products.product_name))_smartwater® Electrolyte Enhanced Water
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,False,6,1.290994,3,0,0,1.5,4,11,...,0,0,0,0,0,0,0,0,0,0
2,1,True,98,2.160247,17,0,11,14.0,7,29,...,0,0,0,0,0,0,0,0,0,0
3,2,False,140,1.581139,30,0,26,28.0,5,14,...,0,0,0,0,0,0,0,0,0,0
7,3,False,222,1.290994,57,0,54,55.5,4,36,...,0,0,0,0,0,0,0,0,0,0
10,4,False,338,1.290994,86,0,83,84.5,4,20,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,762,False,64198,1.290994,16051,0,16048,16049.5,4,18,...,0,0,0,0,0,0,0,0,0,0
997,763,False,64218,1.290994,16056,0,16053,16054.5,4,7,...,0,0,0,0,0,0,0,0,0,0
998,764,False,112518,2.160247,16077,0,16071,16074.0,7,41,...,0,0,0,0,0,0,0,0,0,0


## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [6]:
from collections import Counter
# --- Write your code below ---

# Select top-10 features and return X, y (X.shape = (767, 10)
checked_features = feature_matrix.iloc[:,4:].drop(['SKEW(orders.Unnamed: 0)'],axis=1)
label_column = feature_matrix['label']
corr = {}
for i,feature in enumerate(checked_features.columns):
    corr[feature] = checked_features.iloc[:,i].corr(label_column)
k= Counter(corr)
top_10 = dict(k.most_common(10))

    

### 2.2 Get accuracy and list features

In [7]:
top_10_features = list(top_10.keys())
X = feature_matrix[top_10_features]
y = feature_matrix['label']
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.77


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.4s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
1. Please list three advantages and disadvantages of featuretools.

-> Advantages

1). It reduces development time as there is no manual intervention in feature extraction and engineering process. It can augment the complete end-to-end data science lifecycle.

2). It helps in better prediction performance of the model by choosing only best features.

3). It can adapt to categorical as well as time features which would be otherwise hard to feature engineer.

-> Disadvantages

1). Featuretools does not deal with null values effectively. We have to resort to other methods. 

2). Featuretools only generates features in the statistical domain - Mean, Median, Mode, and other statistical measures. These might not suffice in predicting an approximate prediction. 

3). Domain knowledge is very important in assessing a models features, featuretools doesnt relate to the domain knowledge information. 

2. For those disadvantages you listed above, do you have any idea to improve it?

-> Feature tools can be improved to handle null values by replacing with mean or mode of the feature.

-> Featuretools can work on providing categories of features in accordance with the specific domain by improving package by consulting with domain experts. 

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 7.